In [1]:
import moldesign as mdt
from moldesign import units as u

OpenMM could not be imported; using remote docker container
PDBFixer could not be imported; using remote docker container
PySCF not installed; using remote docker container
Reading configuration from /Users/t_leeday/.moldesign/moldesign.yml


In [2]:
from moldesign.models.lammps_model import LAMMPSPotential
import parmed as med

In [3]:
mol = mdt.from_pdb('1yu8')


In [4]:
mol = mdt.Molecule([res for res in mol.residues if res.type == "protein"])

In [5]:
mdmol = mdt.assign_forcefield(mol)

print "Done parameterizing"



Connecting to docker host at unix://var/run/docker.sock... done


Widget Javascript not detected.  It may not be installed or enabled properly.


Done parameterizing


In [6]:
mdmol.set_energy_model(LAMMPSPotential)


In [7]:
hydrogens = [atom for atom in mdmol.atoms if atom.atnum == 1]
dist_before = []
for atom in hydrogens:
    assert atom.num_bonds ==1
    d = atom.distance(atom.bonded_atoms[0])
    dist_before.append(d)

In [8]:
hello = mdmol.calculate()


Creating system
LAMMPS output is captured by PyLammps wrapper


In [9]:
hello['potential_energy']

<Quantity(-4354.13539812, 'kcalpermol')>

In [10]:
mdmol.set_integrator(mdt.integrators.LAMMPSNvt,
                   timestep=1.0*u.fs,
                   temperature=300.0*u.kelvin,
                   frame_interval=1*u.ps)

In [11]:
traj = mdmol.run(10*u.ps)

Creating system
LAMMPS output is captured by PyLammps wrapper
1.0
fix 1 all nvt temp 300.0 300.0 100.0
fix 2 all shake 0.0001 20 0 m 1.008
Done - integrated "Molecule: Unnamed molecule from OpenMM" from 0.0 fs to 10000.0 fs


In [12]:
import parmed as med
parmedmol = med.load_file('/tmp/tmp.prmtop')

In [13]:
numNonConstrained = 0
for idx, atom in enumerate(hydrogens):
    assert atom.num_bonds ==1
    d = atom.distance(atom.bonded_atoms[0])
    if abs(d-dist_before[idx]).value_in(u.angstrom) > 0.0001:
        numNonConstrained+=1
print numNonConstrained

32


In [14]:
traj.draw()


Widget Javascript not detected.  It may not be installed or enabled properly.
